# Intro to HPC: ClimRR

::: {#fig-climrr}

![](https://www.anl.gov/sites/www/files/2021-09/CPA_RESIZE_Climate%20Resilience%20Images_01_1920x1080.jpg)

[ClimRR](https://disgeoportal.egs.anl.gov/ClimRR/): Empowering individuals,
governments, and organizations to examine simulated future climate conditions
at mid- and end-of-century for a range of climate perils[^img]
:::


## Climate Risk \& Resilience Portal (`ClimRR`)

The [ClimRR Portal](https://www.anl.gov/ccrds/ClimRR) can be viewed online, and
is a great resource for interactive exploration.


Climate change is increasing the complexity, intensity, and frequency of
disasters.

Understanding future climate conditions in cities and towns across the United
States is necessary to prepare for future climate realities.  

To address this requirement, ClimRR[^climrr] — the Climate Risk and Resilience
Portal — empowers individuals, governments, and organizations to examine
simulated future climate conditions at mid- and end-of-century for a range of
climate perils.


<!-- ::: {.grid style="justify-items: start; align-items: center;"}

::: {.g-col-6 style="text-align:left; place-self: center;"}

_Empowering individuals, governments, and organizations to examine simulated
future climate conditions at mid- and end-of-century for a range of climate
perils._

::: {.g-col-6 style="text-align:left; place-self: end;"}

:::

:::
  
::: {.g-col-6 style="align-self: center;"}

::: {#fig-climrr style="text-align:center;"}
![[](https://www.anl.gov/ccrds/ClimRR)](https://www.anl.gov/sites/www/files/2021-09/CPA_RESIZE_Climate%20Resilience%20Images_01_1920x1080.jpg)

[Image Source](https://www.anl.gov/sites/www/files/2021-09/CPA_RESIZE_Climate%20Resilience%20Images_01_1920x1080.jpg)
:::

:::

::: -->

::: {.callout-important title="Finding the data" collapse="true" style="width:100%; place-self: end;"}

- The data needed for this project has already been copied to the NERSC systems and can be found at:

```bash
$ ls /global/cfs/cdirs/m4388/Project2-ClimRR/data/ClimRR/
 AnnualTemperatureMaximum.csv  'ClimRR Metadata and Data Dictionary.pdf'   FireWeatherIndex_Wildfire.csv   GridCellsShapefile.zip   Precipitation_inches_AnnualTotal.csv   SeasonalTemperatureMaximum.csv   WindSpeed.csv
 AnnualTemperatureMinimum.csv   ConsecutiveDayswithNoPrecipitation.csv     GridCells2Shapefile.zip         HeatingDegreeDays.csv    README.txt                             SeasonalTemperatureMinimum.csv
```

the data is also available online and can be downloaded from [ClimRR Data Download
(ANL)](https://anl.box.com/s/hmkkgkrkzxxocfe9kpgrzk2gfc4gizp8)

<!-- - [Counties - United States of America](https://public.opendatasoft.com/explore/dataset/georef-united-states-of-america-county/export/?disjunctive.ste_code&disjunctive.ste_name&disjunctive.coty_code&disjunctive.coty_name&sort=year&dataChart=eyJxdWVyaWVzIjpbeyJjb25maWciOnsiZGF0YXNldCI6Imdlb3JlZi11bml0ZWQtc3RhdGVzLW9mLWFtZXJpY2EtY291bnR5Iiwib3B0aW9ucyI6eyJkaXNqdW5jdGl2ZS5zdGVfY29kZSI6dHJ1ZSwiZGlzanVuY3RpdmUuc3RlX25hbWUiOnRydWUsImRpc2p1bmN0aXZlLmNvdHlfY29kZSI6dHJ1ZSwiZGlzanVuY3RpdmUuY290eV9uYW1lIjp0cnVlLCJzb3J0IjoieWVhciJ9fSwiY2hhcnRzIjpbeyJhbGlnbk1vbnRoIjp0cnVlLCJ0eXBlIjoibGluZSIsImZ1bmMiOiJDT1VOVCIsInNjaWVudGlmaWNEaXNwbGF5Ijp0cnVlLCJjb2xvciI6IiNGRjUxNUEifV0sInhBeGlzIjoieWVhciIsIm1heHBvaW50cyI6IiIsInRpbWVzY2FsZSI6InllYXIiLCJzb3J0IjoiIn1dLCJkaXNwbGF5TGVnZW5kIjp0cnVlLCJhbGlnbk1vbnRoIjp0cnVlfQ%3D%3D) -->
<!---->

:::

[^climrr]: ClimRR was developed by the [Center for Climate Resilience and Decision
Science (CCRDS)](https://www.anl.gov/ccrds) at Argonne National Laboratory in collaboration with AT&T and
the United States Department of Homeland Security’s [Federal Emergency
Management Agency (FEMA)](https://www.fema.gov/). 


[^variables]: [Online Reference for Variables](https://disgeoportal.egs.anl.gov/ClimRR/?page=ClimRR-Data&views=Lack-of-Precipitation)

&nbsp;

> ::: {style="font-size:1.5em; padding-top: -1em!important; margin:auto;"}
> **Project Description**  
> :::
> Are different populations unequally affected  by climate change? This project
> will explore the intersection of climate data and energy justice, providing an
> introductory understanding of data science techniques in assessing disparate
> outcomes on vulnerable or underserved populations in the United States.
> By shedding light on energy disparities, the project aims to contribute to the
> ongoing discussions surrounding energy equity and inform policymakers and
> stakeholders for more inclusive energy initiatives. Energy justice refers to
> the equitable distribution of clean, affordable, and reliable energy resources
> among all communities, regardless of their socioeconomic status or demographic
> characteristics.
> 
> This project will focus on the analysis of climate data and low-income energy
> affordability data across the United States. The analysis will focus on
> examining energy consumption patterns, renewable energy adoption rates, and
> energy affordability across different geographical regions and demographic
> groups.
> 
> To assess disparate outcomes, the project will employ various statistical
> techniques to compare energy-related indicators among vulnerable populations.
> 
> This work may involve analyzing energy access in low-income neighborhoods,
> evaluating the impact of energy policies on marginalized communities, or
> examining the correlation between energy affordability and socioeconomic
> factors.
> 
> The findings of the analysis will be interpreted to gain insights into the
> disparities in energy justice and its impact on vulnerable populations. The
> project aims to identify areas where energy inequalities exist and provide a
> data-driven foundation for advocating for equitable energy policies and
> programs.


::: {.column-page}

## Additional Projects

| **Project (and link project description)**                                                                                                                                                                                                                                         | Institution              | **Lead**                 |
|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |:------------------------:|:------------------------:|
| [AI-Powered Equity Analysis of Renewable Energy Laws](https://shinstitute.org/ai-powered-equity-analysis-of-renewable-energy-laws/)                                                                                                                                           | ANL                      | Murat Keceli                  |
| [Solar Power for Affordable Housing through Computational Design of Low-Cost/High-Efficiency Solar Cells](https://shinstitute.org/solar-power-for-affordable-housing-through-computational-design-of-low-cost-high-efficiency-solar-cells/)                                   | ANL                      | Alvaro Vazquez-Mayagoita      |
| [Energy Justice Analysis of Climate Data](https://shinstitute.org/energy-justice-analysis-of-climate-data/)                                                                                                                                                                   | ANL                      | Sam Foreman                   |
| [Energy Cost for Disadvantaged Populations and Methods of Energy Efficiency and Energy Optimization in Computing Systems](https://shinstitute.org/energy-cost-for-disadvantaged-populations-and-methods-of-energy-efficiency-and-energy-optimization-in-computing-systems/) | NERSC                    | Charles Lively                  |
| [Understanding the Impact of HPC Center Energy Usage on Low-income and Minority Populations](https://shinstitute.org/understanding-the-impact-of-hpc-center-energy-usage-on-low-income-and-minority-populations/)                                                           | NERSC                    |  Charles Lively                 |
| [Power Outages and Inequities in Energy Access for Medically Vulnerable Populations](https://shinstitute.org/power-outages-and-inequities-in-energy-access-for-medically-vulnerable-populations/)                                                                            | OLCF                     | Subil Abraham                  |
| [Socioeconomics of Power Outages and Heatwaves](https://shinstitute.org/socioeconomics-of-power-outages-and-heatwaves/)                                                                                                                                                      | OLCF                     | Suzanne Parete-Koon            |

:::


[^climrr]: ClimRR was developed by the Center for Climate Resilience and Decision
Science (CCRDS) at Argonne National Laboratory in collaboration with AT&T and
the United States Department of Homeland Security’s Federal Emergency
Management Agency (FEMA). Climate change is increasing the complexity,
intensity, and frequency of disasters. Understanding future climate conditions
in cities and towns across the United States is necessary to prepare for future
climate realities.  


[^img]: [Image Source](https://www.anl.gov/sites/www/files/2021-09/CPA_RESIZE_Climate%20Resilience%20Images_01_1920x1080.jpg)

<!-- :::: {} -->
<!---->
<!-- ::: {style="float:left; position:relative; top:0px; padding:30px;"} -->
<!---->
<!-- ![[ClimRR](https://www.anl.gov/ccrds/ClimRR)](https://www.anl.gov/sites/www/files/2021-09/CPA_RESIZE_Climate%20Resilience%20Images_01_1920x1080.jpg) -->
<!---->
<!-- ::: -->
<!---->
<!-- ::: {style="float:right!important; position:relative; top:0px; padding:30px;"} -->
<!---->
<!-- Climate change is increasing the complexity, intensity, and frequency of -->
<!-- disasters. Understanding future climate conditions in cities and towns across -->
<!-- the United States is necessary to prepare for future climate realities. To -->
<!-- address this requirement, ClimRR — the Climate Risk and Resilience Portal — -->
<!-- empowers individuals, governments, and organizations to examine simulated -->
<!-- future climate conditions at mid- and end-of-century for a range of climate -->
<!-- perils.   -->
<!---->
<!-- ClimRR was developed by the Center for Climate Resilience and Decision -->
<!-- Science (CCRDS) at Argonne National Laboratory in collaboration with AT&T and -->
<!-- the United States Department of Homeland Security’s Federal Emergency -->
<!-- Management Agency (FEMA). -->
<!---->
<!-- ::: -->
<!---->
<!-- :::: -->